In [1]:
###importing packages

import scipy as sp
import numpy as np; #importing packages we will use
import matplotlib.pyplot as plt;
import pandas as pd;
pd.options.mode.chained_assignment = None  # default='warn'
import multiprocessing
from multiprocessing.pool import ThreadPool as Pool
from scipy.io import loadmat
import os 
import scipy.linalg as la
import time
from scipy.optimize import minimize
import pickle
import time
from numba import jit
from sklearn.linear_model import LinearRegression
from linearmodels.iv import IV2SLS
import statsmodels.api as sm

In [2]:
os.chdir('G:\\Greg\\full_model')

In [212]:
##importing the plan data and the individual data from STATA

os.chdir('G:\\Greg\\full_model')
individual_data = pd.read_stata(r"individual_data_2019.dta")
individual_data['income_h'] = individual_data['income_h']/100000
individual_data['baseid'] = individual_data['baseid'].astype(str)
individual_data.loc[individual_data['merge_variable'] == '', 'merge_variable'] = 'OO'
plan_data_raw = pd.read_stata(r"planinfo_data_2019.dta")

In [214]:
###droppping columns with missing data (except for annual drog deductible) (I pre-checked most of the data so I don't have
# much missing data)

cols_to_exclude = 'annualdrugdeductible'
cols_to_check = [col for col in plan_data_raw.columns if col != cols_to_exclude]
plan_data = plan_data_raw.dropna(subset=cols_to_check)

In [215]:
##only keeping counties in both MCBS and plan-data

individual_data_counties = individual_data['county_year'].unique()
plan_data_counties = plan_data['county_year'].unique()
usable_counties_1 = np.intersect1d(individual_data_counties,plan_data_counties)

In [218]:
cols_to_drop = ['state', 'county_landscape','monthlyconsolidatedpremiumi', 'annualdrugdeductible',
               'year', 'year_str', 'merge_variable','plan_market_share','contract_plan_id','county_year', 
                'outside_option_share','firm_classification', 'name_merge','firmclassification_encoded','cpi_adjustment_factor']

### getting usable plans (dropping individuals with plans not-in usable_county_years or with plans not in plan_data)

In [220]:
plan_data2 = plan_data[plan_data['county_year'].isin(usable_counties_1)]

In [221]:
###getting all of the plans that are in the plan_level data (and appending the outside option)

all_plans = plan_data2['merge_variable'].unique()
len(all_plans)
all_plans_OO = np.append(all_plans, 'OO')

In [1]:
#keeping plans that are in the usable county plans

individual_data2 = individual_data[individual_data['county_year'].isin(usable_counties_1)]

In [223]:
###drops around 1,000 people whose plan_ids aren't in the merged_data)

individual_data_dropped = individual_data2[individual_data2['merge_variable'].isin(all_plans_OO)]

In [226]:
#individual_data_dropped is all of the individuals
#plan_data2 is all of the data on plans that are not dropped

### next, keep counties with at least three individuals, and two plans

In [227]:
observations_by_county_year = individual_data_dropped.groupby('county_year').size().reset_index(name='individuals_county')
individual_data_dropped = pd.merge(individual_data_dropped, observations_by_county_year, on='county_year')
individual_data_3 = individual_data_dropped[individual_data_dropped['individuals_county']>= 3]

In [2]:
plan_obs_cty_year = plan_data2.groupby('county_year').size().reset_index(name='plans_county')
plan_data3 = pd.merge(plan_data2, plan_obs_cty_year, on='county_year')

In [229]:
##keep county_years of counties with at least two plans
plan_data4 = plan_data3[plan_data3['plans_county']>=2]

In [230]:
###creating plans with usable county years

usable_plan_counties = plan_data4['county_year'].unique()
usable_individual_counties = individual_data_3['county_year'].unique()
usable_county_years = np.intersect1d(usable_plan_counties,usable_individual_counties)

### Doing some data modifications

In [231]:
###getting dummies for income terciles
tercile_dummies = pd.get_dummies(individual_data_3['tercile_income'], prefix = 'income_tercile')
health_dummies = pd.get_dummies(individual_data_3['genhelth'], prefix = 'healthcode')
individual_data_4 = pd.concat([individual_data_3, tercile_dummies, health_dummies], axis=1)

In [233]:
##scaling variable

individual_data_4.loc[individual_data_4['merge_variable'] == '', 'merge_variable'] = 'OO'
individual_data_4['income_h'] = individual_data_4['income_h']/10000
individual_data_4['income_h_sq'] = individual_data_4['income_h']**2
individual_data_4['baseid'] = individual_data_4['baseid'].astype(str)
individual_data_4['h_age'] = individual_data_4['h_age']/100
individual_data_4['h_age_sq'] = individual_data_4['h_age']**2

##modifying plan data 
plan_data4['eff_price'] = plan_data4['eff_price']/100
plan_data4['costsharing_pmpm']= plan_data4['costsharing_pmpm']/100
plan_data4['eff_premium_instrument1'] = plan_data4['eff_premium_instrument1']/100
plan_data4['eff_premium_instrument2'] = plan_data4['eff_premium_instrument2']/100
plan_data4['eff_premium_instrument_border'] = plan_data4['eff_premium_instrument_border']/100
plan_data4['premium_instrument_1'] = plan_data4['premium_instrument_1']/100
plan_data4['premium_instrument_2'] = plan_data4['premium_instrument_2']/100
plan_data4['premium_instrument_border'] = plan_data4['premium_instrument_border']/100
plan_data4['cs_instrument_1'] = plan_data4['cs_instrument_1']/100
plan_data4['cs_instrument_2'] = plan_data4['cs_instrument_2']/100
plan_data4['cs_instrument_border'] = plan_data4['cs_instrument_border']/100

In [234]:
###modifying the sample weights to weight each observation by it's proportion

ever_enrolled_sum = individual_data_4['eeyrswgt'].sum()
individual_data_4['eer_weight'] = individual_data_4['eeyrswgt'].apply(lambda x: x/ever_enrolled_sum)
individual_data_4['weight_scaled'] = individual_data_4['eer_weight'].apply(lambda x: x*len(individual_data_4))
individual_data_4['even_weight'] = 1/(len(individual_data_4))

In [239]:
###adjusting for inflation
monetary_variables = ['costsharing_pmpm', 'eff_premium_instrument1',
       'eff_premium_instrument2', 'premium_instrument_1','premium_instrument_2', 'cs_instrument_1', 'cs_instrument_2',
       'eff_premium_instrument_border', 'premium_instrument_border','cs_instrument_border', 'eff_price']

plan_data4_adjusted = plan_data4[monetary_variables].div(plan_data4['cpi_adjustment_factor'], axis=0)
plan_data4_adjusted.columns = [var + '_adjusted' for var in plan_data4_adjusted.columns]
plan_data5 = pd.concat([plan_data4, plan_data4_adjusted], axis = 1)

#### Getting dummy varibles for star_rating and drug rating status

In [241]:
county_dummies = pd.get_dummies(plan_data5['ssastatecountycode_string'], prefix='county')
year_dummies = pd.get_dummies(plan_data5['year'], prefix='year')
starrating_dummies = pd.get_dummies(plan_data5['star_string'], prefix='rating')
drugbenefit_dummies = pd.get_dummies(plan_data5['drugbenefittype'], prefix = 'drugbenefit')

plan_data5['firm_classification'] = plan_data5['firm_classification'].replace('Blue', 'Blue-Cross-Blue-Shield')
##export plan_data5 to stata to get information

plan_data5['lin_util_init'] = plan_data5['ln_sj_s0']
plan_data5['mean_util_guess'] = plan_data5['ln_sj_s0']

## Code to get data_dictionary

In [243]:
###greating dictionary with the county_year as the key where I have, for each county
# the individual_data from the MCBS and the plan_data from different plan_data files

main_dictionary = {}

for value in usable_county_years:
    temp_data = individual_data_4[individual_data_4['county_year']==value]
    temp_data2 = plan_data5[plan_data5['county_year']==value]
    dict_temp = {value : {'individual_data': temp_data, 'plan_data': temp_data2}}
    main_dictionary.update(dict_temp)

In [244]:
with open('main_dictionary_2019.pkl', 'wb') as fp:
    pickle.dump(main_dictionary, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


#### saving useful variables

In [245]:
with open('useful_variables.pkl', 'wb') as fp:
    pickle.dump(usable_county_years, fp)
    print('variable saved successfully to file')

variable saved successfully to file


### Getting individual and plan data

In [3]:
with open('main_dictionary_2019.pkl', 'rb') as fp:
    main_dictionary = pickle.load(fp)
with open('useful_variables.pkl', 'rb') as fp:
    usable_county_years = pickle.load(fp)

In [5]:
##saving individual_data to use for summary statistics
individual_data_frame = pd.DataFrame()

for county in usable_county_years:
    temp_df_ind = main_dictionary[county]['individual_data']
    individual_data_frame = pd.concat([individual_data_frame, temp_df_ind])

individual_data_frame.to_stata('individual_data_usable_cty.dta')

C:\Users\GES58\AppData\Local\Temp\ipykernel_10044\355933748.py:7: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    income_tercile_1.0   ->   income_tercile_1_0
    income_tercile_2.0   ->   income_tercile_2_0
    income_tercile_3.0   ->   income_tercile_3_0

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  individual_data_frame.to_stata('individual_data_usable_cty.dta')


In [7]:
#saving plan_data to use for summary_statistics
new_data_frame = pd.DataFrame()

for county in usable_county_years:
    temp_df = main_dictionary[county]['plan_data']
    new_data_frame =pd.concat([new_data_frame, temp_df])
    
new_data_frame.to_stata('plan_data_usable_cty.dta')

C:\Users\GES58\AppData\Local\Temp\ipykernel_10044\2213799824.py:7: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    eff_premium_instrument_border_adjusted   ->   eff_premium_instrument_border_ad
    premium_instrument_border_adjusted   ->   premium_instrument_border_adjust

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  new_data_frame.to_stata('plan_data_usable_cty.dta')
